### 1. Objectives

- Quantify the extent of class imbalance (Positive, Negative, Neutral) in the Bangla Sentiment Dataset and its impact on sentiment classification performance.
- Evaluate three lightweight mitigation techniques—Synthetic Minority Oversampling Technique (SMOTE), random undersampling, and weighted loss functions—in improving model accuracy and robustness.
- Assess performance differences across dataset sources (newspapers, social media, blogs) to understand source-specific imbalance effects.

### 2. Hypotheses

- **H1**: Class imbalance in the Bangla Sentiment Dataset reduces sentiment classification performance, particularly for minority classes, as measured by per-class F1-scores.
- **H2**: Mitigation techniques (SMOTE, undersampling, weighted loss) significantly improve model performance compared to baseline models, as measured by weighted F1-score and ROC-AUC.
- **H3**: Mitigation effectiveness varies across sources (newspapers, social media, blogs) due to differences in linguistic style and imbalance patterns.



In [1]:
# Load the dataset
import pandas as pd

# Load the Excel file
df = pd.read_excel('data-source/sentiment_dataset.xlsx', engine='openpyxl')

# Show first 5 rows
print(df.head())

                                               Tense  Label
0  জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে ...    0.0
1  সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান ...    1.0
2  দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সর...    0.0
3          ওনার মতো ব্যর্থ মন্ত্রীর পদত্যাগ করা উচিত    0.0
4                 আল্লাহ তোদের বিচার করবে অপেক্ষা কর    0.0


In [3]:
# Convert to csv for better load speed
df.to_csv('data-source/data.csv', index=False)


# Import all necessary libraries

In [2]:
# Basic data processing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, f1_score,
    classification_report, confusion_matrix
)

# MLflow tracking
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

In [4]:
# Load the csv file
df = pd.read_csv('data-source/data.csv')

## Initial Data Exploration

In [7]:
df.head(5)

,Tense,Label
0,জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে ...,0.0
1,সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান ...,1.0
2,দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সর...,0.0
3,ওনার মতো ব্যর্থ মন্ত্রীর পদত্যাগ করা উচিত,0.0
4,আল্লাহ তোদের বিচার করবে অপেক্ষা কর,0.0


In [10]:
# shape of dataset
print("Shape of dataset: {}".format(df.shape))

Shape of dataset: (9162, 2)


In [12]:
# Display the column names
print("Column names in the dataset:")
print(df.columns)

Column names in the dataset:
Index(['Tense', 'Label'], dtype='object')


In [14]:
# Display the data types of each column
print("Data types of each column:")
print(df.dtypes)

Data types of each column:
Tense     object
Label    float64
dtype: object


In [15]:
# Display information about the dataset
print("Information about the dataset:")
print(df.info())

Information about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9162 entries, 0 to 9161
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Tense   9162 non-null   object 
 1   Label   9154 non-null   float64
dtypes: float64(1), object(1)
memory usage: 143.3+ KB
None


In [18]:
# Check for null values in the dataset
print("Null values in the dataset:")
print(df.isnull().sum())

Null values in the dataset:
Tense    0
Label    8
dtype: int64


In [20]:
# Check for duplicate values in the dataset
print("Duplicate values in the dataset:")
print(df.duplicated().sum())

Duplicate values in the dataset:
1412


In [21]:
# Display summary statistics of the dataset
print("Summary statistics of the dataset:")
print(df.describe())

Summary statistics of the dataset:
             Label
count  9154.000000
mean      0.856565
std       0.860452
min       0.000000
25%       0.000000
50%       1.000000
75%       2.000000
max       2.000000


In [24]:
# Separate categorical and numerical features
categorical_features = df.select_dtypes(include="object").columns
numerical_features = df.select_dtypes(exclude="object").columns
print(categorical_features)
print(numerical_features)

Index(['Tense'], dtype='object')
Index(['Label'], dtype='object')


### Phase 1: Data Exploration and Imbalance Analysis

- **Objective**: Quantify class imbalance and establish baseline characteristics.
- **Steps**:
    1. Load dataset using pandas.
    2. Calculate label distribution (Positive, Negative, Neutral) overall and by source (newspapers, social media, blogs).
    3. Visualize distributions using matplotlib/seaborn (e.g., bar charts, pie charts).
    4. Identify imbalance severity (e.g., ratio of majority to minority classes).

In [32]:
# Display first few rows to verify
print(df.head())

# Check for missing values
print(df.isnull().sum()) 

                                               Tense  Label
0  জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে ...    0.0
1  সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান ...    1.0
2  দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সর...    0.0
3          ওনার মতো ব্যর্থ মন্ত্রীর পদত্যাগ করা উচিত    0.0
4                 আল্লাহ তোদের বিচার করবে অপেক্ষা কর    0.0
Tense    0
Label    8
dtype: int64


In [33]:
# Verify unique tense and labels
print("Tense:", df['Tense'].unique())
print("Lables:", df['Label'].unique()) 

Tense: ['জিনিসপত্রের অতিরিক্ত দাম বৃদ্ধির জন্য এই শহরে জীবন ধারণ করা কঠিন হয়ে যাচ্ছে'
 'সঠিক ভাবে তদারকি করলে এই সমস্যা থেকে পরিত্রান পওয়া সম্ভব'
 'দেশের টাকা যখন বিদেশে চোলে যাচ্ছে তখন দেশের সরকার কী করেন ' ...
 'দয়া করে কেও ফুড পান্ডায় ওয়াডার দিবেন না'
 'আপনাদের কাছে টাকা আছে ভালো কথা  আপনারা রেষ্টুরেন্টে গিয়ে পছন্দসই খাবার অর্ডার করে গরম খাবার খাবেন'
 'এই ঘরবন্দি জীবনে ফুডপান্ডার ভুমিকা সত্যিই প্রসংশনীয় ভালো কাজের জন্যে শুভকামনা']
Lables: [ 0.  1.  2. nan]


In [36]:
# Overall label distribution
overall_distribution = df['Tense'].value_counts()
overall_distribution_percent = df['Tense'].value_counts(normalize=True) * 100

In [37]:
print("Overall Label Distribution (Counts):")
print(overall_distribution)
print("\nOverall Label Distribution (Percentages):")
print(overall_distribution_percent)

Overall Label Distribution (Counts):
Tense
প্রকাশ্যে ফাঁসি দিলে এমন কাজ আর কেউ করতে সাহস পেতোনা                             9
এই হাসপাতালের সাথে যারা জড়িত আছে সবাই কে ফাঁসি দিতে হবে                          6
ডেসটিনি থেকে যারা শিক্ষা নিতে পারলোনা তাদের কপালে এমনটা হওয়াই উচিৎ               6
বাংলাদেশের মানুষ খুব লোভী তার জন্যই বারবার এমন হয়                                6
বর্তমানে বাংলাদেশের যতগুলো ই কমার্স সাইট আছে সবগুলোই বাটপার                      5
                                                                                ..
বর্তমানে দেশের জনগন মনে হয় সবকিছুর জন্য শুধু পুলিশকে দায়ী করছে                   1
দুর্নীতি বন্ধ ও দমন করতে পারলে জিনিষপত্রদের দাম অর্ধেক করা সম্ভব                 1
দুর্নীতিবাজদের দুর্নীতির কারণে দেশের আজকে বেহাল অবস্থা                           1
এমন উন্নয়ন আমরা চাই না যেই উন্নয়নের কারণে মানুষের ভোগান্তি অনেক বাড়ে          1
এই ঘরবন্দি জীবনে ফুডপান্ডার ভুমিকা সত্যিই প্রসংশনীয় ভালো কাজের জন্যে শুভকামনা    1
Name: count, Length: 7584, dtype: int64

Ove

In [38]:
 # label distribution
label_distribution = df.groupby('Label')['Tense'].value_counts().unstack(fill_value=0)
label_distribution_percent = df.groupby('Label')['Tense'].value_counts(normalize=True).unstack(fill_value=0) * 100

print("\nLabel-Specific Data Distribution (Counts):")
print(label_distribution)
print("\nLabel-Specific Label Distribution (Percentages):")
print(label_distribution_percent)


Label-Specific Data Distribution (Counts):
Tense  আমাদের দেশের মানুষ এত বোকা কেন তারা কেন এইসব এজেন্টদের টাকা দেয়  \
Label                                                                    
0.0                                                    5                 
1.0                                                    0                 
2.0                                                    0                 

Tense  এই সব ভুয়া ই কমার্স কোম্পানিগুলো বারবার মানুষকে ঠকাচ্ছে  \
Label                                                             
0.0                                                    5          
1.0                                                    0          
2.0                                                    0          

Tense  এগুলো সমাজের নিম্নমানের ও অসভ্য সমাজের লোক  \
Label                                               
0.0                                             5   
1.0                                             0   
2.0                                  